In [1]:
pip install tensorflow

In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install cmake

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2

In [6]:
%%cmd
where python

Microsoft Windows [Version 10.0.22621.3593]
(c) Microsoft Corporation. All rights reserved.

C:\Users\archa>where python
C:\Users\archa\anaconda3\anakonda 2\python.exe
C:\Users\archa\AppData\Local\Programs\Python\Python310\python.exe
C:\Users\archa\AppData\Local\Microsoft\WindowsApps\python.exe

C:\Users\archa>

In [7]:
%%cmd
python --version

Microsoft Windows [Version 10.0.22621.3593]
(c) Microsoft Corporation. All rights reserved.

C:\Users\archa>python --version
Python 3.11.7

C:\Users\archa>

In [8]:
%%cmd
pip install "C:\Users\archa\Downloads\Dlib_Windows_Python3.x-main\Dlib_Windows_Python3.x-main\dlib-19.24.1-cp311-cp311-win_amd64.whl"

Microsoft Windows [Version 10.0.22621.3593]
(c) Microsoft Corporation. All rights reserved.

C:\Users\archa>pip install "C:\Users\archa\Downloads\Dlib_Windows_Python3.x-main\Dlib_Windows_Python3.x-main\dlib-19.24.1-cp311-cp311-win_amd64.whl"
Processing c:\users\archa\downloads\dlib_windows_python3.x-main\dlib_windows_python3.x-main\dlib-19.24.1-cp311-cp311-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

C:\Users\archa>

In [9]:
%%cmd
pip install face-recognition

Microsoft Windows [Version 10.0.22621.3593]
(c) Microsoft Corporation. All rights reserved.

C:\Users\archa>pip install face-recognition

C:\Users\archa>

In [10]:
import tensorflow as tf
import numpy as np
import cv2

import dlib
import pickle

In [11]:
def get_model():
    backbone = tf.keras.applications.EfficientNetB2(
        input_shape=(96, 96, 3),
        include_top=False,
        weights=None
    )
    model = tf.keras.Sequential([
        backbone,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')
    ])
    return model

In [12]:
from keras.models import Sequential
from keras.layers import Dense
import keras.saving

# Step 1: Define and Compile Your Model
def get_model():
    model = Sequential()
    model.add(Dense(64, input_shape=(784,), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

# Create and compile your model
model = get_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 2: Save the Entire Model in .keras Format
model.save(r"C:\Users\archa\Downloads\best_model.keras")

# Step 3: Load the Saved Model and Verify
from keras.models import load_model
loaded_model = load_model(r"C:\Users\archa\Downloads\best_model.keras")

# Verify the loaded model
loaded_model.summary()


C:\Users\archa\anaconda3\anakonda 2\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\archa\anaconda3\anakonda 2\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │          50,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 165,152 (645.13 KB)

 Trainable params: 55,050 (215.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 110,102 (430.09 KB)

In [13]:
import pickle

# Define a function to load a pickled object
def load_object(name):
    with open(name, "rb") as pickle_obj:
        obj = pickle.load(pickle_obj)
    return obj

# Load the LabelEncoder
Le = load_object("C:/Users/archa/Downloads/LabelEncoder.pck")

# Now you can use `Le` as your LabelEncoder object
print(Le)  # Example of how you might use the loaded object


LabelEncoder()


In [14]:
def ProcessImage(image):
    image = tf.convert_to_tensor(image)
    image = tf.image.resize(image , [96 , 96] , method="bilinear")
    image = tf.expand_dims(image , 0)
    return image

def RealtimePrediction(image , model, encoder_):
    prediction = model.predict(image)
    prediction = np.argmax(prediction , axis = 1)
    return encoder_.inverse_transform(prediction)[0]

def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

In [ ]:
VideoCapture = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()

while True :
    
    ret , frame = VideoCapture.read() 
    
    if not ret :
        break

    gray = cv2.cvtColor( frame , cv2.COLOR_BGR2GRAY)

    rects = detector(gray , 0)

    if len(rects) >= 1 :
        for rect in rects :
            (x , y , w , h) = rect_to_bb(rect)
            img = gray[y-10 : y+h+10 , x-10 : x+w+10]
            
            if img.shape[0] == 0 or img.shape[1] == 0 :
                cv2.imshow("Frame", frame)
                
            else :
                img = cv2.cvtColor(img , cv2.COLOR_GRAY2RGB)
                img = ProcessImage(img)
                out = RealtimePrediction(img , model , Le)
                cv2.rectangle(frame, (x, y), (x+w, y+h),(0, 255, 0), 2)
                z = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, str(out), (x, z), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
                
        cv2.imshow("Frame", frame)
            
    else :
        cv2.imshow("Frame", frame)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
VideoCapture.release()
cv2.destroyAllWindows()